In [1]:
#from huggingface_hub import notebook_login
#from kaggle_secrets import UserSecretsClient
#user_secrets = UserSecretsClient()
#hf_token = user_secrets.get_secret("huggingface")
#
#notebook_login(hf_token)

In [2]:
%%capture

!pip install faiss-gpu -q

!pip install deep-translator -q

!pip install langdetect -q

!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip uninstall -y torch torchvision torchaudio xformers
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
!pip install sentence_transformers peft

!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [3]:
import torch

import numpy as np

import matplotlib.pyplot as plt

import transformers

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

from sentence_transformers import SentenceTransformer

import faiss

from torch.utils.data import Dataset, DataLoader

import torch.nn as nn

from nltk import sent_tokenize

import nltk

from deep_translator import GoogleTranslator

from langdetect import detect

import pickle

import transformers

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

from peft import prepare_model_for_kbit_training
from peft import LoraConfig, TaskType, get_peft_model

from sklearn.model_selection import train_test_split

from unsloth import FastLanguageModel

import json

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
!rm -rf prikols
!git clone https://github.com/bivba/prikols

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Cloning into 'prikols'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 77 (delta 29), reused 34 (delta 16), pack-reused 19 (from 2)
Receiving objects: 100% (77/77), 42.90 MiB | 39.79 MiB/s, done.
Resolving deltas: 100% (33/33), done.


In [5]:
token = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', device='cuda')

device = 'cuda' if torch.cuda.is_available() else 'cpu'



client = GoogleTranslator(source='ru', target='en')

text = open('/kaggle/working/prikols/context.txt').read()

In [6]:
#model_name = "unsloth/Meta-Llama-3.1-8B-bnb-4bit"

torch.cuda.empty_cache()

max_seq_length = 4096

dtype = None

load_in_4bit = True



model, tokenizer = FastLanguageModel.from_pretrained(

    model_name = "bivba/quant_4bit",

    max_seq_length = max_seq_length,

    dtype = dtype,

    load_in_4bit = load_in_4bit

)

==((====))==  Unsloth 2024.12.8: Fast Qwen2 patching. Transformers: 4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Unsloth 2024.12.8 patched 48 layers with 48 QKV layers, 48 O layers and 48 MLP layers.


In [7]:
nltk.download('punkt_tab')

def split_document(document, chunk_size=256, overlap=32):

    sentences = sent_tokenize(document)

    chunks = []

    current_chunk = []

    current_length = 0



    for sentence in sentences:

        sentence_length = len(sentence)

        if current_length + sentence_length <= chunk_size:

            current_chunk.append(sentence)

            current_length += sentence_length

        else:

            chunks.append(" ".join(current_chunk))

            current_chunk = current_chunk[-overlap:] + [sentence]

            current_length = sum(len(s) for s in current_chunk)



    if current_chunk:

        chunks.append(" ".join(current_chunk))



    return chunks

[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [8]:
chunks = split_document(text)

encoded = token.encode(chunks, return_numpy=True)

Batches:   0%|          | 0/486 [00:00<?, ?it/s]

In [9]:
def add_index(encoded):

  num_vectors = len(encoded)

  dim = encoded.shape[1]

  faiss_index = faiss.IndexFlatL2(dim)



  faiss_index.add(np.array(encoded, dtype=np.float32))

In [10]:
faiss_index = faiss.read_index('prikols/context (1).faiss')

In [11]:
def find_context(query, k=3):

  if detect(query) == 'ru':

    query = client.translate(query)

  q_emb = token.encode([query])

  q_emb = q_emb.astype(np.float32)

  _, indices = faiss_index.search(np.array(q_emb), k)

  retrieved_chunks = [chunks[i] for i in indices[0]]

  context = ' '.join(retrieved_chunks)

  return context

In [12]:
model = FastLanguageModel.get_peft_model(

    model,

    r = 16,

    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",

                      "gate_proj", "up_proj", "down_proj",],

    lora_alpha = 16,

    lora_dropout = 0, # Supports any, but = 0 is optimized

    bias = "none",    # Supports any, but = "none" is optimized

    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!

    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context

    random_state = 3407,

    use_rslora = False,  # We support rank stabilized LoRA

    loftq_config = None, # And LoftQ

)

Unsloth: Already have LoRA adapters! We shall skip this step.


In [13]:
def load_data(type:str='full data'):

    qas = None
    
    if type == 'qa':
        questions = []
        s = ''
        with open('prikols/q/quest.txt', encoding='utf-8') as f:
            for line in f:
                if line != '\n':
                    s += line
                else:
                    questions.append(s[3:])
                    s = ''
        questions.append(s[3:])
        print(len(questions))
        answers = []
        st = ''
        with open('prikols/a/asnw_text.txt', encoding='utf-8') as f:
            for line in f:
                if line != '\n':
                    st += line
                else:
                    answers.append(st[3:])
                    st = ''
        answers.append(st[3:])
        print(len(answers))
        
        qa = [{'question': questions[i], 'answer': answers[i]} for i in range(len(questions))]
        
        json.dump(qa, open('prikols/qa.json', 'w', encoding='utf-8'), indent=2, ensure_ascii=False)
        
        qas = load_dataset('json', data_files='prikols/qa.json', split='train')

    if type == 'full data':

        qas = load_dataset('json', data_files='prikols/qa.json', split='train')

    if type == 'feedback':

        qas = load_dataset('json', data_files='prikols/feedback.json', split='train')

    return qas

In [14]:
from datasets import load_dataset

In [15]:
question_prompt = """You are an AI assistaint helping with answering questions about quantum physics and quantum computer science. Below is a Question ,write a response that appropriately answers the request.



### Question:

{}
### Answer:

{}"""

def map_data(dataset, type:str='qa'):  
    
    EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
    
    def formatting_prompts_func(examples):
    
        instructions = [client.translate(example) for example in examples['quesion']] if (all([detect(example) for example in examples['question']]) == 'ru') else examples['question']
        inputs       = [find_context(instruction) for instruction in instructions]
        outputs      = [client.translate(example) for example in examples['answer']] if (all([detect(example) for example in examples['answer']]) == 'ru') else examples['answer']
    
        texts = []
    
        for instruction, input, output in zip(instructions, inputs, outputs):
    
            # Must add EOS_TOKEN, otherwise your generation will go on forever!
    
            text = question_prompt.format(instruction, input, output) + EOS_TOKEN
    
            texts.append(text)
    
        return { "text" : texts, }

    def feedback_based_prompt(example):

        question = client.translate(example['question'])

        example['question'] = question_prompt.format(question, '') + EOS_TOKEN

        example['accepted'] = client.translate(example['accepted']) + EOS_TOKEN

        example['rejected'] = client.translate(example['rejected']) + EOS_TOKEN

        return example

    if type == 'qa':

        return dataset.map(formatting_prompts_func, batched=True)

    if type == 'feedback':

        return dataset.map(feedback_based_prompt)
    


qas = load_data(type='feedback')

qas = map_data(qas, type='feedback')

qas = qas.rename_columns({'question': 'prompt', 'accepted': 'chosen'})

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/75 [00:00<?, ? examples/s]

In [16]:
qas[9]['prompt']

'You are an AI assistaint helping with answering questions about quantum physics and quantum computer science. Below is a Question ,write a response that appropriately answers the request.\n\n\n\n### Question:\n\nWhat is the "model of non-interacting" electrons when describing a multi-electron atom? Choose the correct answer \\\\\n1) electron spins are not taken into account when specifying the Hamiltonian \\\\\n2) atomic terms are determined from the solution of the Schrödinger equation for the hydrogen atom \\\\\n3) electrons do not interact with each other \\\\\n4) electrons do not interact with the nucleus of the atom and with each other \\\\\n### Answer:\n\n<|endoftext|>'

**DPO Trainer**

In [17]:
from unsloth import PatchDPOTrainer
PatchDPOTrainer()

In [18]:
from transformers import TrainingArguments
from trl import DPOTrainer, DPOConfig
from unsloth import is_bfloat16_supported
from copy import deepcopy


dpo_trainer = DPOTrainer(
    
    model = model,
    
    ref_model = None,
    
    args = DPOConfig(
        
        per_device_train_batch_size = 1,
        
        gradient_accumulation_steps = 4,
        
        warmup_ratio = 0.1,
        
        num_train_epochs = 5,
        
        learning_rate = 5e-6,
        
        fp16 = not is_bfloat16_supported(),
        
        bf16 = is_bfloat16_supported(),
        
        logging_steps = 1,
        
        optim = "adamw_8bit",
        
        weight_decay = 0.0,
        
        lr_scheduler_type = "linear",
        
        seed = 42,
        
        output_dir = "outputs",
        
        report_to = "none", # Use this for WandB etc
        
    ),
    
    beta = 0.1,
    
    train_dataset = qas,
    
    # eval_dataset = raw_datasets["test"],
    
    tokenizer = tokenizer,
    
    max_length = 512,
    
    max_prompt_length = 512,
    
)

Extracting prompt from train dataset:   0%|          | 0/75 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/75 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/75 [00:00<?, ? examples/s]

In [20]:
dpo_trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 75 | Num Epochs = 5
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 4
\        /    Total batch size = 4 | Total steps = 90
 "-____-"     Number of trainable parameters = 68,812,800


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / rejected,logps / chosen,logits / rejected,logits / chosen
1,2.206300,0.054689,0.646905,0.500000,-0.592217,-243.041641,-142.562027,-0.059503,0.172456
2,0.262100,2.971041,-3.959291,0.750000,6.930332,-244.476944,-138.817932,0.438893,0.162768
3,5.724200,-1.968487,2.517681,0.250000,-4.486168,-192.353119,-126.269684,0.301847,0.684618
4,3.092500,-1.300197,0.558558,0.750000,-1.858755,-237.736038,-156.497894,-0.266343,0.292250
5,1.574100,0.343137,-1.018733,0.750000,1.361870,-274.092987,-153.615631,0.156235,0.694602
6,8.119000,-1.369043,6.180636,0.250000,-7.549679,-247.832321,-117.849907,0.202107,-0.065803
7,2.684900,-0.553334,1.340205,0.500000,-1.893539,-278.011078,-125.283585,-0.703956,0.414598
8,3.770500,-1.176760,1.132730,0.500000,-2.309491,-176.469040,-133.665863,0.646260,1.096055
9,0.074100,-0.393355,-5.434069,1.000000,5.040714,-384.803894,-116.376740,-1.041013,0.324267
10,3.245600,-2.229934,-2.613094,0.750000,0.383159,-490.567749,-123.837280,-0.755217,0.008125


TrainOutput(global_step=90, training_loss=1.0018616499168274, metrics={'train_runtime': 2285.5455, 'train_samples_per_second': 0.164, 'train_steps_per_second': 0.039, 'total_flos': 0.0, 'train_loss': 1.0018616499168274, 'epoch': 4.8})

**SFT Trainer**

In [16]:
from trl import SFTTrainer

from transformers import TrainingArguments

from unsloth import is_bfloat16_supported



trainer = SFTTrainer(

    model = model,

    tokenizer = tokenizer,

    train_dataset = qas,

    dataset_text_field = "text",

    max_seq_length = max_seq_length,

    dataset_num_proc = 2,

    packing = False, # Can make training 5x faster for short sequences.

    args = TrainingArguments(

        per_device_train_batch_size = 2,

        gradient_accumulation_steps = 4,

        warmup_steps = 5,

        #max_steps = 60,

        num_train_epochs = 5, # For longer training runs!

        learning_rate = 2e-4,

        fp16 = not is_bfloat16_supported(),

        bf16 = is_bfloat16_supported(),

        logging_steps = 1,

        optim = "adamw_8bit",

        weight_decay = 0.01,

        lr_scheduler_type = "linear",

        seed = 42,

        output_dir = "outputs",

        report_to = "none", # Use this for WandB etc

    ),

)

Map (num_proc=2):   0%|          | 0/90 [00:00<?, ? examples/s]

In [17]:
#@title Show current memory stats

gpu_stats = torch.cuda.get_device_properties(0)

start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)

max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)

print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")

print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
10.656 GB of memory reserved.


In [18]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 90 | Num Epochs = 5
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 55
 "-____-"     Number of trainable parameters = 68,812,800


Step,Training Loss
1,0.835700
2,0.802900
3,0.781300
4,0.759400
5,0.772100
6,0.743900
7,0.860800
8,0.814200
9,0.820000
10,0.652000


**Testing inference**

In [23]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

inputs = tokenizer(

[

    question_prompt.format(

        client.translate(" Кубит - это суперпозиция двух квантовых состояний $c_0|0> + c_1|1>$, где $c_1, c_2$ - комплексные числа. Сколько действительных параметров нужно задать, чтобы определить конкретный кубит. \\\\\n1) 2 \\\\\n2) 3 \\\\\n3) 1 \\\\\n4) 4 \\\\\n"), # instruction

        find_context(" Кубит - это суперпозиция двух квантовых состояний $c_0|0> + c_1|1>$, где $c_1, c_2$ - комплексные числа. Сколько действительных параметров нужно задать, чтобы определить конкретный кубит. \\\\\n1) 2 \\\\\n2) 3 \\\\\n3) 1 \\\\\n4) 4 \\\\\n"), # input

        "", # output - leave this blank for generation!

    )

], return_tensors = "pt").to("cuda")



outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True, pad_token_id = tokenizer.eos_token_id)

tokenizer.batch_decode(outputs)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

["You are an AI assistaint helping with answering questions about quantum physics and quantum computer science. Below is a Question with a supportive context. Write a response that appropriately answers the request.\n\n\n\n### Question:\n\nA qubit is a superposition of two quantum states $c_0|0> + c_1|1>$, where $c_1, c_2$ are complex numbers. How many real parameters need to be specified to define a specific qubit. \\\\\n1) 2 \\\\\n2) 3 \\\\\n3) 1 \\\\\n4) 4 \\\\\n\n\n\n### Context:\n\n[label=0., ref=0]\n2. **General Two-Qubit States** In our system of two qubits, the state space is four-dimensional with basis states \\(\\ket{00}\\), \\(\\ket{01}\\), \\(\\ket{10}\\), and \\(\\ket{11}\\). This allows us to construct a general two-qubit state as a linear combination of these basis states: \\[\\ket{\\psi}=\\sum_{i,j\\in\\{0,1\\}}c_{ij}\\ket{ij}=c_{00}\\ket{00}+c_{01}\\ket{01}+c_ {10}\\ket{10}+c_{11}\\ket{11}.\\] (6.14)\n\nHere, \\(c_{ij}\\) are complex coefficients such that \\(\\sum|c_{

In [21]:
!git lfs install

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Git LFS initialized.


In [22]:
!git clone https://huggingface.co/bivba/quant_4bit

Cloning into 'quant_4bit'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 57 (delta 22), reused 0 (delta 0), pack-reused 3 (from 1)
Unpacking objects: 100% (57/57), 1.74 MiB | 5.67 MiB/s, done.


In [24]:
model.push_to_hub("bivba/quant_4bit", token = "...", commit_message='DPO trained', commit_description='DPO trained') # Online saving

tokenizer.push_to_hub("bivba/quant_4bit", token = "...", commit_message='DPO trained', commit_description='DPO trained') # Online saving

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/275M [00:00<?, ?B/s]

Saved model to https://huggingface.co/bivba/quant_4bit


In [ ]:
model.push_to_hub_merged("bivba/quant", tokenizer, save_method = "merged_16bit", token = "...")